In [66]:
import os
import torch
import tqdm
import numpy as np
import pandas as pd
import torchvision.models as models
import torchvision.transforms as transforms
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR
from sklearn.metrics import precision_recall_fscore_support
import rasterio  
import torch.optim as optim

In [67]:
def set_seed(seed):
    # Set seed
    torch.manual_seed(seed)

    np.random.seed(seed)

    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(21)

In [68]:
# Check if cuda is available
device = torch.device("cpu")

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("DEVICE = CUDA")

DEVICE = CUDA


In [ ]:
def construct_patch_path(data_path, survey_id):
    """Construct the patch file path based on plot_id as './CD/AB/XXXXABCD.tiff'"""
    path = data_path
    for d in (str(survey_id)[-2:], str(survey_id)[-4:-2]):
        path = os.path.join(path, d)

    path = os.path.join(path, f"{survey_id}.tiff")

    return path

In [ ]:
def quantile_normalize(band, low=2, high=98):
    sorted_band = np.sort(band.flatten())
    quantiles = np.percentile(sorted_band, np.linspace(low, high, len(sorted_band)))
    normalized_band = np.interp(band.flatten(), sorted_band, quantiles).reshape(band.shape)
    
    min_val, max_val = np.min(normalized_band), np.max(normalized_band)
    
    # Prevent division by zero if min_val == max_val
    if max_val == min_val:
        return np.zeros_like(normalized_band, dtype=np.float32)  # Return an array of zeros

    # Perform normalization (min-max scaling)
    return ((normalized_band - min_val) / (max_val - min_val)).astype(np.float32)

In [69]:
class Sentinel_TrainDataset(Dataset):
    def __init__(self, data_dir, metadata, transform=None):
        self.transform = transform
        self.data_dir = data_dir
        self.metadata = metadata
        self.metadata = self.metadata.dropna(subset="speciesId").reset_index(drop=True)
        self.metadata['speciesId'] = self.metadata['speciesId'].astype(int)
        self.label_dict = self.metadata.groupby('surveyId')['speciesId'].apply(list).to_dict()
        
        self.metadata = self.metadata.drop_duplicates(subset="surveyId").reset_index(drop=True)

    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, idx):
        survey_id = self.metadata.surveyId[idx]
        species_ids = self.label_dict.get(survey_id, [])  # Get list of species IDs for the survey ID
        label = torch.zeros(num_classes)  # Initialize label tensor
        for species_id in species_ids:
            label_id = species_id
            label[label_id] = 1  # Set the corresponding class index to 1 for each species
        
        # Read TIFF files (multispectral bands)
        tiff_path = construct_patch_path(self.data_dir, survey_id)
        with rasterio.open(tiff_path) as dataset:
            image = dataset.read(out_dtype=np.float32)  # Read all bands
            image = np.array([quantile_normalize(band) for band in image])  # Apply quantile normalization

        image = np.transpose(image, (1, 2, 0))  # Convert to HWC format
        image = self.transform(image)

        return image, label, survey_id
    
class Sentinel_TestDataset(Sentinel_TrainDataset):
    def __init__(self, data_dir, metadata, transform=None):
        self.transform = transform
        self.data_dir = data_dir
        self.metadata = metadata
        
    def __getitem__(self, idx):
        survey_id = self.metadata.surveyId[idx]
        
        # Read TIFF files (multispectral bands)
        tiff_path = construct_patch_path(self.data_dir, survey_id)
        with rasterio.open(tiff_path) as dataset:
            image = dataset.read(out_dtype=np.float32)  # Read all bands
            image = np.array([quantile_normalize(band) for band in image])  # Apply quantile normalization

        image = np.transpose(image, (1, 2, 0))  # Convert to HWC format
        
        image = self.transform(image)
        return image, survey_id

In [70]:
class Climate_TrainDataset(Dataset):
    def __init__(self, data_dir, metadata, subset, transform=None):
        self.subset = subset
        self.transform = transform
        self.data_dir = data_dir
        self.metadata = metadata
        self.metadata = self.metadata.dropna(subset="speciesId").reset_index(drop=True)
        self.metadata['speciesId'] = self.metadata['speciesId'].astype(int)
        self.label_dict = self.metadata.groupby('surveyId')['speciesId'].apply(list).to_dict()
        
        self.metadata = self.metadata.drop_duplicates(subset="surveyId").reset_index(drop=True)

    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, idx):
        survey_id = self.metadata.surveyId[idx]
        sample = torch.load(os.path.join(self.data_dir, f"GLC25-PA-{self.subset}-bioclimatic_monthly_{survey_id}_cube.pt"), weights_only=True)
        species_ids = self.label_dict.get(survey_id, [])  # Get list of species IDs for the survey ID
        label = torch.zeros(num_classes)  
        for species_id in species_ids:
            label_id = species_id
            label[label_id] = 1  # Set the corresponding class index to 1 for each species

        # Ensure the sample is in the correct format for the transform
        if isinstance(sample, torch.Tensor):
            sample = sample.permute(1, 2, 0)  # Change tensor shape from (C, H, W) to (H, W, C)
            sample = sample.numpy()  

        if self.transform:
            sample = self.transform(sample)

        return sample, label, survey_id
    
class Climate_TestDataset(Climate_TrainDataset):
    def __init__(self, data_dir, metadata, subset, transform=None):
        self.subset = subset
        self.transform = transform
        self.data_dir = data_dir
        self.metadata = metadata
        
    def __getitem__(self, idx):
        survey_id = self.metadata.surveyId[idx]
        sample = torch.load(os.path.join(self.data_dir, f"GLC25-PA-train-bioclimatic_monthly_{survey_id}_cube.pt"), weights_only=True)
        
        if isinstance(sample, torch.Tensor):
            sample = sample.permute(1, 2, 0)  # Change tensor shape from (C, H, W) to (H, W, C)
            sample = sample.numpy()

        if self.transform:
            sample = self.transform(sample)

        return sample, survey_id

In [71]:
class Landsat_TrainDataset(Dataset):
    def __init__(self, data_dir, metadata, subset, transform=None):
        self.subset = subset
        self.transform = transform
        self.data_dir = data_dir
        self.metadata = metadata
        self.metadata = self.metadata.dropna(subset="speciesId").reset_index(drop=True)
        self.metadata['speciesId'] = self.metadata['speciesId'].astype(int)
        self.label_dict = self.metadata.groupby('surveyId')['speciesId'].apply(list).to_dict()
        self.metadata = self.metadata.drop_duplicates(subset="surveyId").reset_index(drop=True)


    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, idx):
        survey_id = self.metadata.surveyId[idx]
        sample = torch.nan_to_num(torch.load(os.path.join(self.data_dir, f"GLC25-PA-{self.subset}-landsat-time-series_{survey_id}_cube.pt"), weights_only=True))
        species_ids = self.label_dict.get(survey_id, [])  # Get list of species IDs for the survey ID
        label = torch.zeros(num_classes)  # Initialize label tensor
        for species_id in species_ids:
            label_id = species_id
            label[label_id] = 1  # Set the corresponding class index to 1 for each species

        if isinstance(sample, torch.Tensor):
            sample = sample.permute(1, 2, 0)  # Change tensor shape from (C, H, W) to (H, W, C)
            sample = sample.numpy()  # Convert tensor to numpy array
            #print(sample.shape)

        if self.transform:
            sample = self.transform(sample)

        return sample, label, survey_id
    
class Landsat_TestDataset(Dataset):
    def __init__(self, data_dir, metadata, subset, transform=None):
        super().__init__()
        self.data_dir  = data_dir
        self.metadata  = metadata.reset_index(drop=True)
        self.subset    = subset
        self.transform = transform

    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, idx):
        survey_id = self.metadata.surveyId[idx]

        fname = f"GLC25-PA-{self.subset}-landsat-time-series_{survey_id}_cube.pt"
        path  = os.path.join(self.data_dir, fname)
        sample = torch.nan_to_num(torch.load(path, weights_only=True))

        if isinstance(sample, torch.Tensor):
            sample = sample.permute(1,2,0).numpy()

        if self.transform:
            sample = self.transform(sample)

        return sample, survey_id

# Advanced Model-1 (Sentinel, Landsat, and BioClimate)

### Data and configuration

In [72]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

CLIMATE_DIR = "/fs/scratch/PAS2985/group_23/BioclimTimeSeries/cubes/PA-train"
LANDSAT_DIR = "/fs/scratch/PAS2985/group_23/SateliteTimeSeries-Landsat/cubes/PA-train/"
SENTINEL_DIR = "/fs/scratch/PAS2985/group_23/SatelitePatches/PA-train"

CLIMATE_TRAIN_DIR = "/fs/scratch/PAS2985/group_23/BioclimTimeSeries/cubes/PA-train"
CLIMATE_TEST_DIR = "/fs/scratch/PAS2985/group_23/BioclimTimeSeries/cubes/PA-train"
CLIMATE_META_TRAIN = "/fs/scratch/PAS2985/group_23/training_data.csv"
CLIMATE_META_TEST = "/fs/scratch/PAS2985/group_23/test_data.csv"

LANDSAT_TRAIN_DIR = "/fs/scratch/PAS2985/group_23/SateliteTimeSeries-Landsat/cubes/PA-train/"
LANDSAT_TEST_DIR = "/fs/scratch/PAS2985/group_23/SateliteTimeSeries-Landsat/cubes/PA-train/"
LANDSAT_META_TRAIN = "/fs/scratch/PAS2985/group_23/training_data.csv"
LANDSAT_META_TEST = "/fs/scratch/PAS2985/group_23/test_data.csv"

SENTINEL_TRAIN_DIR = "/fs/scratch/PAS2985/group_23/SatelitePatches/PA-train"
SENTINEL_TEST_DIR = "/fs/scratch/PAS2985/group_23/SatelitePatches/PA-train"
SENTINEL_META_TRAIN = "/fs/scratch/PAS2985/group_23/training_data.csv"
SENTINEL_META_TEST = "/fs/scratch/PAS2985/group_23/test_data.csv"

NUM_CLASSES = 11255
BATCH_SIZE = 32
LR = 1e-4
EPOCHS = 20
POS_WEIGHT = 1.0  # Positive weight factor

Device: cuda


### Dataloaders

In [73]:
num_classes = NUM_CLASSES

climate_transform = transforms.Compose([transforms.ToTensor()])
landsat_transform = transforms.Compose([transforms.ToTensor()])
sentinel_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5,)*4, std=(0.5,)*4)
])

# load metadata
meta_train = pd.read_csv("/fs/scratch/PAS2985/group_23/training_data.csv")
meta_test = pd.read_csv("/fs/scratch/PAS2985/group_23/test_data.csv")

# Initiate datasets
cl_train_ds = Climate_TrainDataset(CLIMATE_TRAIN_DIR, cl_meta_tr, subset="train", transform=climate_transform)
cl_test_ds = Climate_TestDataset(CLIMATE_TRAIN_DIR, cl_meta_te, subset="train", transform=climate_transform)

ls_train_ds = Landsat_TrainDataset(LANDSAT_TRAIN_DIR, ls_meta_tr, subset="train", transform=landsat_transform)
ls_test_ds = Landsat_TestDataset(LANDSAT_TRAIN_DIR, ls_meta_te, subset="train", transform=landsat_transform)

se_train_ds = Sentinel_TrainDataset(SENTINEL_TRAIN_DIR, se_meta_tr, transform=sentinel_transform)
se_test_ds = Sentinel_TestDataset(SENTINEL_TRAIN_DIR, se_meta_te, transform=sentinel_transform)

In [74]:
cl_train_ds = Climate_TrainDataset(CLIMATE_DIR, meta_train, subset="train", transform=climate_transform)
cl_test_ds = Climate_TestDataset(CLIMATE_DIR, meta_test, subset="train", transform=climate_transform)

ls_train_ds = Landsat_TrainDataset(LANDSAT_DIR, meta_train, subset="train", transform=landsat_transform)
ls_test_ds = Landsat_TestDataset(LANDSAT_DIR, meta_test, subset="train", transform=landsat_transform)

se_train_ds = Sentinel_TrainDataset(SENTINEL_DIR, meta_train, transform=sentinel_transform)
se_test_ds = Sentinel_TestDataset (SENTINEL_DIR, meta_test, transform=sentinel_transform)

print("Modalities train sizes:", len(cl_train_ds), len(ls_train_ds), len(se_train_ds))
print("Modalities  test sizes:", len(cl_test_ds), len(ls_test_ds), len(se_test_ds))

Modalities train sizes: 62290 62290 62290
Modalities  test sizes: 223031 223031 223031


In [75]:
# CombinedDataset: returns ((cl_img, ls_img, se_img), label, survey_id)
from torch.utils.data import Dataset

class CombinedTrainDataset(Dataset):
    def __init__(self, ds_cl, ds_ls, ds_se):
        assert len(ds_cl)==len(ds_ls)==len(ds_se), "modalities must align"
        self.cl, self.ls, self.se = ds_cl, ds_ls, ds_se

    def __len__(self):
        return len(self.cl)

    def __getitem__(self, idx):
        # each returns (img, label, survey_id)
        x_cl, y, id_cl = self.cl[idx]
        x_ls, _, _     = self.ls[idx]
        x_se, _, _     = self.se[idx]
        return (x_cl, x_ls, x_se), y, id_cl

class CombinedTestDataset(Dataset):
    def __init__(self, ds_cl, ds_ls, ds_se):
        assert len(ds_cl)==len(ds_ls)==len(ds_se), "modalities must align"
        self.cl, self.ls, self.se = ds_cl, ds_ls, ds_se

    def __len__(self):
        return len(self.cl)

    def __getitem__(self, idx):
        # each returns (img, survey_id)
        x_cl, id_cl = self.cl[idx]
        x_ls, id_ls = self.ls[idx]
        x_se, id_se = self.se[idx]
        # sanity check that IDs match
        assert id_cl == id_ls == id_se
        return (x_cl, x_ls, x_se), id_cl

In [76]:
train_ds = CombinedTrainDataset(cl_train_ds, ls_train_ds, se_train_ds)
test_ds = CombinedTestDataset(cl_test_ds, ls_test_ds, se_test_ds)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

print("Batches per epoch:", len(train_loader))

Batches per epoch: 1947


In [9]:
class CombinedModel(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.climate = models.resnet18(weights=None)
        self.climate.conv1 = nn.Conv2d(4, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.climate.maxpool = nn.Identity()
        self.climate.fc = nn.Identity()

        self.landsat = models.resnet18(weights=None)
        self.landsat.conv1 = nn.Conv2d(6, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.landsat.maxpool = nn.Identity()
        self.landsat.fc = nn.Identity()

        self.sentinel = models.resnet18(weights=None)
        self.sentinel.conv1 = nn.Conv2d(4, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.sentinel.maxpool = nn.Identity()
        self.sentinel.fc = nn.Identity()

        # Final classifier
        self.classifier = nn.Linear(512*3, NUM_CLASSES)

    def forward(self, inputs):
        x_cl, x_ls, x_se = inputs
        f_cl = self.climate(x_cl)
        f_ls = self.landsat(x_ls)
        f_se = self.sentinel(x_se)
        # concatenate and classify
        cat = torch.cat([f_cl, f_ls, f_se], dim=1)
        return self.classifier(cat)

model = CombinedModel(NUM_CLASSES).to(device)
print(model)

Batches per epoch: 1947
CombinedModel(
  (climate): ResNet(
    (conv1): Conv2d(4, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): Identity()
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): R

In [10]:
optimizer = optim.AdamW(model.parameters(), lr=LR)
scheduler = CosineAnnealingLR(optimizer, T_max=EPOCHS, verbose=True)

def criterion_fn(outputs, targets):
    pos_weight = targets * POS_WEIGHT
    return nn.BCEWithLogitsLoss(pos_weight=pos_weight)(outputs, targets)

for epoch in range(1, EPOCHS+1):
    model.train()
    total_loss = 0.0

    for (x_cl, x_ls, x_se), labels, _ in train_loader:
        x_cl, x_ls, x_se, labels = x_cl.to(device), x_ls.to(device), x_se.to(device), labels.to(device)
        optimizer.zero_grad()
        preds = model((x_cl, x_ls, x_se))
        loss = criterion_fn(preds, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * x_cl.size(0)

    scheduler.step()
    avg_loss = total_loss / len(train_loader.dataset)
    print(f"Epoch {epoch}/{EPOCHS} — Loss: {avg_loss:.4f}")

model.eval()
all_preds, all_ids = [], []

/users/PAS2985/patwardhan24/.local/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/20 — Loss: 0.0093
Epoch 2/20 — Loss: 0.0047
Epoch 3/20 — Loss: 0.0044
Epoch 4/20 — Loss: 0.0042
Epoch 5/20 — Loss: 0.0040
Epoch 6/20 — Loss: 0.0038
Epoch 7/20 — Loss: 0.0036
Epoch 8/20 — Loss: 0.0034
Epoch 9/20 — Loss: 0.0033
Epoch 10/20 — Loss: 0.0031
Epoch 11/20 — Loss: 0.0029
Epoch 12/20 — Loss: 0.0028
Epoch 13/20 — Loss: 0.0027
Epoch 14/20 — Loss: 0.0026
Epoch 15/20 — Loss: 0.0025
Epoch 16/20 — Loss: 0.0024
Epoch 17/20 — Loss: 0.0023
Epoch 18/20 — Loss: 0.0023
Epoch 19/20 — Loss: 0.0022
Epoch 20/20 — Loss: 0.0022


  0%|          | 0/6970 [00:00<?, ?it/s]


FileNotFoundError: Caught FileNotFoundError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/users/PAS2985/patwardhan24/.local/lib/python3.9/site-packages/torch/utils/data/_utils/worker.py", line 349, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
  File "/users/PAS2985/patwardhan24/.local/lib/python3.9/site-packages/torch/utils/data/_utils/fetch.py", line 52, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/users/PAS2985/patwardhan24/.local/lib/python3.9/site-packages/torch/utils/data/_utils/fetch.py", line 52, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/tmp/slurmtmp.587255/ipykernel_3629020/3478093453.py", line 11, in __getitem__
    x_cl, y, id_cl = self.cl[idx]
  File "/tmp/slurmtmp.587255/ipykernel_3629020/2366138481.py", line 46, in __getitem__
    sample = torch.load(os.path.join(self.data_dir, f"GLC25-PA-{self.subset}-bioclimatic

In [11]:
torch.save(model.state_dict(), "combined_model_weights.pth")

In [77]:
all_preds, all_ids = [], []
model.eval()

with torch.no_grad():
    for (x_cl, x_ls, x_se), survey_id in tqdm.tqdm(test_loader):
        x_cl, x_ls, x_se = x_cl.to(device), x_ls.to(device), x_se.to(device)
        logits = model((x_cl, x_ls, x_se))
        probs = torch.sigmoid(logits).cpu().numpy()

        top25 = np.argsort(-probs, axis=1)[:, :25]
        all_preds.append(top25)
        all_ids.extend(survey_id.numpy())

all_preds = np.vstack(all_preds)
pred_str = [" ".join(map(str, row)) for row in all_preds]
submission = pd.DataFrame({'surveyId': all_ids, 'predictions': pred_str})
submission.to_csv("combined_submission.csv", index=False)

100%|██████████| 6970/6970 [39:34<00:00,  2.94it/s]
